## Importing required libraries

In [4]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [5]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium 


## Obtaining and filtering data

In [6]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

In [8]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
df_neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_neighborhoods = df_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
df_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
df_neighborhoods.shape

(306, 4)

In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [13]:
manhattan_data = df_neighborhoods[df_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
map_manhattan = folium.Map(location=[latitude, longitude])
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Obtaining venues for each neighborhood using the Foursquare API

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

In [38]:
CLIENT_ID = 'Hidden'
CLIENT_SECRET = 'Hidden'
ACCESS_TOKEN = 'Hidden'
VERSION = '20180604'

In [17]:
LIMIT = 30
radius = 500

In [18]:
manhattan_venues = getNearbyVenues(names=list(manhattan_data['Neighborhood']),latitudes=manhattan_data['Latitude'],longitudes=manhattan_data['Longitude'],radius = radius)

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [19]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


## Onehot Encoding venue types, grouping similar venues, and normalizing values

In [20]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,

In [21]:
gyms,health,sports,restaurants = [[] for i in range(4)]

In [22]:
for col in list(manhattan_onehot.columns):
    if 'gym' in col.lower():
        gyms.append(col)
    elif 'health' in col.lower():
        health.append(col)
    elif 'sport' in col.lower():
        sports.append(col)
    elif 'restaurant' in col.lower():
        restaurants.append(col)
print(gyms)
print(health)
print(sports)
print(restaurants)

['Boxing Gym', 'Climbing Gym', 'Gym', 'Gym / Fitness Center', 'Gym Pool']
['Health & Beauty Service', 'Health Food Store']
['Sporting Goods Shop', 'Sports Bar', 'Sports Club']
['African Restaurant', 'American Restaurant', 'Argentinian Restaurant', 'Asian Restaurant', 'Australian Restaurant', 'Austrian Restaurant', 'Caribbean Restaurant', 'Chinese Restaurant', 'Cuban Restaurant', 'Czech Restaurant', 'Dim Sum Restaurant', 'Dumpling Restaurant', 'Ethiopian Restaurant', 'Falafel Restaurant', 'Filipino Restaurant', 'French Restaurant', 'German Restaurant', 'Greek Restaurant', 'Hawaiian Restaurant', 'Himalayan Restaurant', 'Hotpot Restaurant', 'Indian Restaurant', 'Italian Restaurant', 'Japanese Curry Restaurant', 'Japanese Restaurant', 'Jewish Restaurant', 'Korean Restaurant', 'Latin American Restaurant', 'Mediterranean Restaurant', 'Mexican Restaurant', 'Middle Eastern Restaurant', 'Molecular Gastronomy Restaurant', 'Moroccan Restaurant', 'New American Restaurant', 'Peruvian Restaurant', '

In [23]:
others = ['Farmers Market']

In [24]:
manhattan_onehot['gyms'] = manhattan_onehot[gyms].sum(axis=1)
manhattan_onehot['restaurants'] = manhattan_onehot[restaurants].sum(axis=1)
df_manhattan = manhattan_onehot[['Neighborhood','gyms','restaurants','Health Food Store','Sporting Goods Shop','Farmers Market']]

In [25]:
df_manhattan = df_manhattan.groupby('Neighborhood').sum().reset_index()
df_manhattan.head()

,Neighborhood,gyms,restaurants,Health Food Store,Sporting Goods Shop,Farmers Market
0,Battery Park City,1,1,0,0,0
1,Carnegie Hill,3,6,0,0,0
2,Central Harlem,1,8,0,0,0
3,Chelsea,0,10,0,0,0
4,Chinatown,0,9,0,0,0


In [26]:
df_manhattan_max_vals = df_manhattan[['gyms','restaurants','Health Food Store','Sporting Goods Shop','Farmers Market']].max()
df_manhattan_max_vals.head()

gyms                    5
restaurants            15
Health Food Store       1
Sporting Goods Shop     3
Farmers Market          1
dtype: int64

In [27]:
df_manhattan[['gyms','restaurants','Health Food Store','Sporting Goods Shop','Farmers Market']] = df_manhattan[['gyms','restaurants','Health Food Store','Sporting Goods Shop','Farmers Market']]/df_manhattan_max_vals

In [28]:
df_manhattan.head()

,Neighborhood,gyms,restaurants,Health Food Store,Sporting Goods Shop,Farmers Market
0,Battery Park City,0.2,0.066667,0.0,0.0,0.0
1,Carnegie Hill,0.6,0.400000,0.0,0.0,0.0
2,Central Harlem,0.2,0.533333,0.0,0.0,0.0
3,Chelsea,0.0,0.666667,0.0,0.0,0.0
4,Chinatown,0.0,0.600000,0.0,0.0,0.0


## Clustering

In [29]:
kclusters = 5
clustering_df = df_manhattan.drop('Neighborhood', axis = 1)
kmeans = KMeans(n_clusters=kclusters,random_state=1).fit(clustering_df)
kmeans.labels_[0:10]

array([1, 1, 0, 0, 0, 1, 4, 0, 0, 1], dtype=int32)

In [30]:
clustering_df['labels'] = kmeans.labels_

In [31]:
resultant_df = clustering_df.groupby('labels').mean()
resultant_df

,gyms,restaurants,Health Food Store,Sporting Goods Shop,Farmers Market
labels,,,,,
0,0.063158,0.673684,0.0,0.000000,0.0
1,0.550000,0.433333,0.0,0.000000,0.0
2,0.100000,0.233333,0.0,0.833333,0.0
3,0.160000,0.386667,0.0,0.000000,1.0
4,0.900000,0.300000,1.0,0.166667,0.0


#### An estimate of the expected number of each venue type in the clusters

In [32]:
resultant_df*df_manhattan_max_vals

,gyms,restaurants,Health Food Store,Sporting Goods Shop,Farmers Market
labels,,,,,
0,0.315789,10.105263,0.0,0.0,0.0
1,2.750000,6.500000,0.0,0.0,0.0
2,0.500000,3.500000,0.0,2.5,0.0
3,0.800000,5.800000,0.0,0.0,1.0
4,4.500000,4.500000,1.0,0.5,0.0


## Combining clusterig results with a map

In [33]:
nbhd_data = manhattan_venues.groupby('Neighborhood').mean()[['Neighborhood Latitude','Neighborhood Longitude']].reset_index()

In [34]:
manhattan_merged = nbhd_data
manhattan_merged['label'] = clustering_df['labels']
manhattan_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,label
0,Battery Park City,40.711932,-74.016869,1
1,Carnegie Hill,40.782683,-73.953256,1
2,Central Harlem,40.815976,-73.943211,0
3,Chelsea,40.744035,-74.003116,0
4,Chinatown,40.715618,-73.994279,0


In [35]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Neighborhood Latitude'], manhattan_merged['Neighborhood Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Neighbourhoods to be recommended to the client

In [36]:
neighborhoods_identified = manhattan_merged[(manhattan_merged['label']==2) | (manhattan_merged['label']==4)]

In [37]:
neighborhoods_identified

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,label
6,Clinton,40.759101,-73.996119,4
16,Lenox Hill,40.768113,-73.958860,4
23,Midtown,40.754691,-73.981669,2
29,Soho,40.722184,-74.000657,2
